In [ ]:
from models import NetworkedRENs, REN, RNNModel
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os

os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
from os.path import dirname, join as pjoin
import torch
from torch import nn

In [ ]:
dtype = torch.float
device = torch.device("cpu")

In [ ]:
# Import Data
folderpath = os.getcwd()
filepath = pjoin(folderpath, 'input_3.mat')#'dataset_sysID_3tanks.mat')
data_in = scipy.io.loadmat(filepath)
filepath = pjoin(folderpath, 'output_Q_3.mat')
data_out = scipy.io.loadmat(filepath)
filepath = pjoin(folderpath, 'subsystems.mat')
data_sub = scipy.io.loadmat(filepath)

filepath = pjoin(folderpath, 'denormalize.mat')
data_max = scipy.io.loadmat(filepath)

# Extract data from dictionary
maxTrit, maxTdel = data_max['maxTrit'], data_max['maxTman']
Toutass_t, Toutass_v, Toutchill_t, Toutchill_v = data_sub['Toutass_train'], data_sub['Toutass_val'], data_sub['Toutchillers_train'], data_sub['Toutchillers_val']
dExp, yExp, dExp_val, yExp_val, time__, buildtot, buildtot_val = data_in['dExp'], data_out['yExp'], \
    data_in['dExp_val'], data_out['yExp_val'], data_in['time__'], data_out['buildtotnorm'], data_out['buildtotnorm_val']
nExp = yExp.size

t = time__

t_end = t.size

#initialize the model

ny = np.shape(yExp[0,-1])[1]
nd = np.shape(dExp[0,-1])[1]

#t = np.arange(0, np.size(dExp[0, 0], 1) * Ts-Ts, Ts)
#t_end = yExp[0, 0].shape[1] - 1

for exp in range(nExp):
    y_exp = yExp[0,exp]
    d_exp = dExp[0,exp]
    plt.figure(figsize=(4 * 2, 4))
    for out in range(ny):
        plt.subplot(2, 2, out+1)
        plt.plot(t[14000:16500], y_exp[14000:16500,out])
        plt.title(r"Water level h%i "%out + r"in experiment %i"%(exp+1))
    plt.subplot(2, 2, ny+1)
    plt.plot(t[14000:16500], d_exp[14000:16500,1])
    plt.title(r"v in experiment %i"%(exp+1))
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1,
                        bottom=0.1, 
                        right=0.9, 
                        top=0.9, 
                        wspace=0.4, 
                        hspace=0.4)
plt.show()

In [ ]:
# TRAIN OF NETWORKED RENs
epochs = 150
t_end = 2500

torch.manual_seed(2)
N = 3 # Number of interconnected systems

n = torch.tensor([2, 2, 2])  # input dimensions
p = torch.tensor([1, 1, 1])  # output dimensions

n_xi = np.array([5, 5, 5]) # nel paper n1, numero di stati
l = np.array([5, 5, 5])  # nel paper q, dimension of the square matrix D11 -- number of _non-linear layers_ of the RE

#alpha = 0.6
#beta = 0.4

#Muy = torch.cat((torch.tensor([[0, alpha, beta], [1, 0, 0], [1, 0, 0]]), torch.zeros(3,3)), dim=0)
#Muy = Muy.float()

Mud = torch.cat((torch.zeros(3,3), torch.eye(3)), dim=0)
#Mey = torch.tensor([[0, alpha, beta], [1, 0, 0]])

# Define the system
RENsys = NetworkedRENs(N, Mud, n, p, n_xi, l)

# Define Loss function
MSE = nn.MSELoss()

# Define Optimization method
learning_rate = 1.0e-1
optimizer = torch.optim.Adam(RENsys.parameters(), lr=learning_rate)
optimizer.zero_grad()

LOSS = np.zeros(epochs)
loss = 0

for epoch in range(epochs):
    if epoch == epochs - epochs / 2:
        learning_rate = 1.0e-2
        optimizer = torch.optim.Adam(RENsys.parameters(), lr=learning_rate)
    if epoch == epochs - epochs / 6:
        learning_rate = 1.0e-3
        optimizer = torch.optim.Adam(RENsys.parameters(), lr=learning_rate)
    optimizer.zero_grad()
    loss = 0
    for exp in range(nExp - 1):
        xi = []
        y = torch.cat((torch.from_numpy(yExp[0, exp][14000:16500,0]).float().to(device).unsqueeze(1),
                       torch.from_numpy(Toutass_t[exp*30240 +14000:exp*30240 + 16500]).float().to(device),
                       torch.from_numpy(Toutchill_t[exp*30240 +14000:exp*30240 + 16500]).float().to(device)), dim=1)
        y = y.T
        yRENm = torch.randn(3,t_end , device=device, dtype=dtype)
        yRENm[0,:] = y[0,:]
        for j in range(N):
            xi.append(torch.randn(RENsys.r[j].n, device=device, dtype=dtype))
        d = torch.cat((torch.from_numpy(buildtot[exp*30240 +14000:exp*30240 + 16500]).float().to(device),
                       torch.from_numpy(dExp[0, exp][14000:16500,-1]).float().to(device).unsqueeze(1),
                       torch.from_numpy(dExp[0, exp][14000:16500,-2]).float().to(device).unsqueeze(1)), dim=1)
        d = d.T
        xi = torch.cat(xi)
        for t in range(1, t_end):
            yRENm[:, t], xi = RENsys(t, d[:, t - 1], xi)

        loss = loss + MSE(yRENm[:, 0:yRENm.size(1)], y[:, 0:t_end + 1])
        # ignorare da loss effetto condizione iniziale

    loss = loss / nExp
    loss.backward()
    # loss.backward(retain_graph=True)

    optimizer.step()

    print(f"Epoch: {epoch + 1} \t||\t Loss: {loss}")
    for net in range(N):
        print(f"L2 gain REN%i"%net+":%.1f"%RENsys.r[net].gamma)
    LOSS[epoch] = loss
    
    # Or save a checkpoint with optimizer and other details
    torch.save({
        'epoch': epoch,
        'model_state_dict': RENsys.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, f'better_epoch_{epoch+1}.pth')




In [ ]:
import torch

# Load 
checkpoint = torch.load('checkpoint_epoch_218.pth')

t_end = 2500
nExp = 1

N = 3 # Number of interconnected systems

n = torch.tensor([2, 2, 2])  # input dimensions
p = torch.tensor([1, 1, 1])  # output dimensions

n_xi = np.array([5, 5, 5]) # nel paper n1, numero di stati
l = np.array([5, 5, 5])  # nel paper q, dimension of the square matrix D11 -- number of _non-linear layers_ of the RE

alpha = 0.6
beta = 0.4

Muy = torch.cat((torch.tensor([[0, alpha, beta], [1, 0, 0], [1, 0, 0]]), torch.zeros(3,3)), dim=0)
Muy = Muy.float()

Mud = torch.cat((torch.zeros(3,3), torch.eye(3)), dim=0)
Mey = torch.tensor([[0, alpha, beta], [1, 0, 0]])

# Define the system
RENsys = NetworkedRENs(N, Muy, Mud, Mey, n, p, n_xi, l)

# Define Loss function
MSE = nn.MSELoss()
# Restore the model state
RENsys.load_state_dict(checkpoint['model_state_dict'])

# If needed, restore the optimizer state (for resuming training)
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Retrieve the last epoch and loss
last_epoch = checkpoint['epoch']
last_loss = checkpoint['loss']

print(f"Resuming from Epoch {last_epoch + 1} with Loss: {last_loss}")

# Ensure the model is in evaluation mode for validation
RENsys.eval()

# Perform validation
with torch.no_grad():
    validation_loss = 0
    for exp in range(nExp - 1):
        xi = []
        yval = torch.cat((torch.from_numpy(yExp_val[0, exp][14000:16500, 0]).float().to(device).unsqueeze(1),
                       torch.from_numpy(Toutass_v[exp * 30240 + 14000:exp * 30240 + 16500]).float().to(device),
                       torch.from_numpy(Toutchill_v[exp * 30240 + 14000:exp * 30240 + 16500]).float().to(device)), dim=1)
        yval = yval.T
        yRENm_val = torch.randn(3, t_end, device=device, dtype=torch.float)
        yRENm_val[0, :] = yval[0, :]
        
        for j in range(N):
            xi.append(torch.randn(RENsys.r[j].in_features, device=device, dtype=torch.float))
            
        dval = torch.cat((torch.from_numpy(buildtot_val[exp * 30240 + 14000:exp * 30240 + 16500]).float().to(device),
                       torch.from_numpy(dExp_val[0, exp][14000:16500, -1]).float().to(device).unsqueeze(1),
                       torch.from_numpy(dExp_val[0, exp][14000:16500, -2]).float().to(device).unsqueeze(1)), dim=1)
        dval = dval.T
        xi = torch.cat(xi)
        
        for t in range(1, t_end):
            yRENm_val[:, t], xi = RENsys(t, dval[:, t - 1], xi)

        validation_loss += MSE(yRENm_val[:, 0:yRENm_val.size(1)], yval[:, 0:t_end + 1])

    validation_loss = validation_loss / nExp
    print(f"Validation Loss: {validation_loss}")
